# Intelligent Element Design

The idea of the intelligent element is to automate the creation of Keras models and data generation for complex nested structures.

In [1]:
from keras import Model
import keras.layers as L

def get_children_sum_last_output_shapes(children_ie):
    '''
    Retrieves the sum of the last dimension of shape of children

    This is usually number of dimensions of RNN or number of channels of images,
    useful to build model
    '''
    if not isinstance(children_ie, list):
        children_ie=[children_ie]
    return sum([x.model.output_shape[-1] for x in children_ie])

def getMaxShape(x):
    #print(len(x))
    if isinstance(x,np.ndarray):
        return x.shape
    elif isinstance(x, list):
        lstshapes = [getMaxShape(e) for e in x]
        maxshape = np.max(lstshapes, axis=0)
        if maxshape[0]>0:
            maxshape = np.concatenate([np.array([len(x)]), maxshape])
        else:
            maxshape = np.array([len(x)])
        return maxshape
    return np.array([0])

def identity_fcn(x):
    return x

class IntelligentElement:
    '''
    Implements IntelligentElement, a base class that can be used to automate Keras model creation
    for complex nested structures
    '''
    
    def __init__(self, data, model, input_shape, preprocess_function = None, children_ie = None, name=None):
        '''
        Initializes the IntelligentElement instance with its data, which must be a list.
        
        All data passed to children IntelligentElements should be a list with the same number of elements
        and matching information. If a dynamic axis is found (input_shape[k] = None for some k), the vectors are 
        padded with zeroes until they get to the max length
        
        data        - list of samples that will be handled by this IE. If there is no model, it can contain empty elements
        children_ie - IEs that handle nested structures
        
        preprocess_function - function that is applied to each element of data list in order 
                              to retrieve neural network ready data as numpy arrays
        
        model - None if there is no data associated, otherwise a Keras Model
        '''
        
        assert isinstance(data, list), 'data should be a list of samples to be handled by this IntelligentElement'
        
        if model is not None:
            assert isinstance(model, Model), 'model should be a Keras model'
            #the shape requirement is that children output matches parent input shape
            #assert len(model.output_shape) == 2, 'model output shape should have length 2'

        if children_ie is not None:
            if not isinstance(children_ie, list):
                children_ie=[children_ie]

            for c in children_ie:
                assert isinstance(c, IntelligentElement), 'children_ie must contain only IntelligentElement'
                assert len(c.data) == len(data), 'length of data vector must be the same for parents and children'
        
        if preprocess_function is None:
            self.preprocess_function = identity_fcn
        else:
            self.preprocess_function = preprocess_function
            
        self.name=name
        self.data = data
        self.input_shape = input_shape
        self.children_ie = children_ie
        self.model = model
        
        if model is not None:
            self.model.name = 'm_{}'.format(name)

    def retrieve_model_inputs_outputs(self):
        inps=[]
        outs=[]
        
        if self.children_ie is not None:
            for c in self.children_ie:
                cmodel, cinp, cout = c.retrieve_model_inputs_outputs()
                inps += cinp
                outs.append(cout)
                
        if self.model is not None:
            inp = L.Input(self.input_shape, name='inp_{}'.format(self.name))
            inps.append(inp)
            outs.append(inp)
            
            #print([q.shape for q in outs])
            if len(outs) > 1:
                o = L.Concatenate()(outs)
            else:
                o = outs[0]
            #print(o.shape)
            outs = [self.model(o)]
            
        if len(outs) > 1:
            o = L.Concatenate()(outs)
        else:
            o = outs[0]
        
        ret_model = Model(inputs = inps, outputs = o, name = self.name)
        
        return ret_model, inps, o
    
    
    def get_batch(self, indices):
        '''
        Retrieves a batch of data as requested in indices
        '''
        inps=[]
        batch_data = [self.data[i] for i in indices]

        if self.children_ie is not None:
            for c in self.children_ie:
                cinp = c.get_batch(indices)
                inps += cinp
        
        if self.model is not None:
            cur_inps = [self.preprocess_function(x) for x in batch_data]
            
            #if there are no dynamic axes, we are done. if not, we need to pad
            shapes = np.array([getMaxShape(x) for x in cur_inps])
            maxshape = np.max(shapes,axis=0)
            
            #print(maxshape)
            padded_inp = np.zeros( (len(indices),*maxshape) )
            
            #print('Padded shape: {}'.format(padded_inp.shape))
            
            
            recursiveFill(padded_inp, cur_inps, ())
            
            #cur_n=0
            #for vec in cur_inps:
            #    ss = ( (cur_n,) )
            #    for p in vec.shape:
            #        ss += (slice(0,p),)
                    
                #print(ss)
                #print(np.array(vec))
                #print(padded_inp.shape)
                
            #    padded_inp[ss] = vec
            #    cur_n+=1
            
            inps.append(padded_inp)
            
            
        return inps

C:\Users\dougl_000\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def recursiveFill(padded_input, vec, cur_slice):
    #print(cur_slice)
    ss = cur_slice
    if isinstance(vec, np.ndarray):
        for p in vec.shape:
            ss += (slice(0,p),)        
        padded_input[ss] = vec
    elif isinstance(vec, list):
        cur_n=0
        for v in vec:
            ss=cur_slice + ((cur_n,))
            recursiveFill(padded_input, v, ss)
            cur_n+=1

# Sample synthesized use

In [3]:
import numpy as np
import pandas as pd
regions_id = {'countryside' : 0, 'central' : 1, 'innerside' : 2}
def genMonthDetails():
    ndays = np.random.randint(4,7)
    ans=[]
    for i in range(ndays):
        ans.append({'day' : i, 'raw_amount' : np.random.randint(400,7000)})
    return ans
    
def genClientData():
    regions = ['countryside', 'central', 'innerside']
    
    ans = {'income' : np.random.randint(10,50), 'age' : np.random.randint(20,70), 
           'residence_region' : regions[np.random.randint(len(regions))] }
    
    ans['last_transactions'] = []
    nlast = np.random.randint(3,10)
    for i in range(nlast):
        last_transaction = {'month' : i, 'value':np.random.randint(1,5)}
        last_transaction['details'] = genMonthDetails()
        ans['last_transactions'].append(last_transaction)
    
    ans['picture'] = np.ones((128,128,3))
    
    return ans

clientdata = [genClientData() for i in range(20)]

In [4]:
pd.DataFrame(clientdata).head()

,age,income,last_transactions,picture,residence_region
0,38,18,"[{'month': 0, 'value': 2, 'details': [{'day': ...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",central
1,61,45,"[{'month': 0, 'value': 4, 'details': [{'day': ...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",countryside
2,26,24,"[{'month': 0, 'value': 1, 'details': [{'day': ...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",countryside
3,32,20,"[{'month': 0, 'value': 2, 'details': [{'day': ...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",central
4,27,27,"[{'month': 0, 'value': 2, 'details': [{'day': ...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",countryside


In [5]:
picture_data = [x['picture'] for x in clientdata]
picture_shape = (128,128,3)
inp=L.Input(picture_shape)
x=L.Conv2D(10,(3,3), activation='relu', padding='same')(inp)
x=L.Flatten()(x)
x=L.Dense(5,activation='relu')(x)
picture_model = Model(inputs=inp, outputs=x)
print('Original model')
picture_model.summary()

picture_ie = IntelligentElement(picture_data, picture_model, picture_shape, name='picture_ie')

m, ii, oo = picture_ie.retrieve_model_inputs_outputs()
print('\n\nRetrieved model')
m.summary()

Original model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 10)      280       
_________________________________________________________________
flatten_1 (Flatten)          (None, 163840)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 819205    
Total params: 819,485
Trainable params: 819,485
Non-trainable params: 0
_________________________________________________________________


Retrieved model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp_picture_ie (InputLayer)  (None, 128, 128, 3)       0         
_____________________________________

In [6]:
picture_ie.get_batch([0,4,5,8,9])[0].shape

(5, 128, 128, 3)

In [7]:
details_data = [[y['details'] for y in x['last_transactions']] for x in clientdata]
details_shape = (None, None, 3)
def details_preproc_function(x):
    #receives a list of dictionaries with keys 'day' and 'raw_amount'
    #append a 1 to indicate that these values were not padded
    ans = []
    for item in x:
        monthdetails = []
        for detaildata in item:
            monthdetails.append([detaildata['day']/30, detaildata['raw_amount']/10000, 1])
        ans.append(np.array(monthdetails))
    
    #handle case when list is empty
    if len(ans) == 0:
        ans.append(np.array([[0,0,0]]))
    return ans

inp = L.Input(details_shape)
x = L.TimeDistributed(L.LSTM(64))(inp)
x = L.TimeDistributed(L.Dense(8, activation='relu'))(x)
details_model = Model(inputs=inp, outputs=x)
print('Original model')
details_model.summary()

details_ie = IntelligentElement(details_data, details_model, details_shape, 
                                    preprocess_function=details_preproc_function, name='details_ie')

m, ii, oo = details_ie.retrieve_model_inputs_outputs()
print('\n\nRetrieved model')
m.summary()

Original model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 64)          17408     
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 8)           520       
Total params: 17,928
Trainable params: 17,928
Non-trainable params: 0
_________________________________________________________________


Retrieved model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp_details_ie (InputLayer)  (None, None, None, 3)     0         
_________________________________________________________________
m_details_ie (Model)         (None, None, 8)           17928     
Total params: 17,928
Trainable params: 

In [8]:
details_data[0]

[[{'day': 0, 'raw_amount': 5008},
  {'day': 1, 'raw_amount': 2933},
  {'day': 2, 'raw_amount': 2952},
  {'day': 3, 'raw_amount': 5474},
  {'day': 4, 'raw_amount': 2725},
  {'day': 5, 'raw_amount': 4500}],
 [{'day': 0, 'raw_amount': 1698},
  {'day': 1, 'raw_amount': 6141},
  {'day': 2, 'raw_amount': 2421},
  {'day': 3, 'raw_amount': 1094},
  {'day': 4, 'raw_amount': 3572}],
 [{'day': 0, 'raw_amount': 5109},
  {'day': 1, 'raw_amount': 2405},
  {'day': 2, 'raw_amount': 4809},
  {'day': 3, 'raw_amount': 1251}],
 [{'day': 0, 'raw_amount': 1788},
  {'day': 1, 'raw_amount': 1979},
  {'day': 2, 'raw_amount': 1346},
  {'day': 3, 'raw_amount': 2265},
  {'day': 4, 'raw_amount': 5894},
  {'day': 5, 'raw_amount': 1959}],
 [{'day': 0, 'raw_amount': 1355},
  {'day': 1, 'raw_amount': 1169},
  {'day': 2, 'raw_amount': 1608},
  {'day': 3, 'raw_amount': 3579},
  {'day': 4, 'raw_amount': 2785},
  {'day': 5, 'raw_amount': 3631}],
 [{'day': 0, 'raw_amount': 1735},
  {'day': 1, 'raw_amount': 870},
  {'day': 

In [9]:
details_preproc = details_preproc_function(details_data[1])
details_preproc

[array([[0.        , 0.6415    , 1.        ],
        [0.03333333, 0.3793    , 1.        ],
        [0.06666667, 0.1965    , 1.        ],
        [0.1       , 0.2072    , 1.        ],
        [0.13333333, 0.6103    , 1.        ],
        [0.16666667, 0.2764    , 1.        ]]),
 array([[0.        , 0.1382    , 1.        ],
        [0.03333333, 0.4353    , 1.        ],
        [0.06666667, 0.4004    , 1.        ],
        [0.1       , 0.6575    , 1.        ]]),
 array([[0.        , 0.6726    , 1.        ],
        [0.03333333, 0.6829    , 1.        ],
        [0.06666667, 0.121     , 1.        ],
        [0.1       , 0.4021    , 1.        ],
        [0.13333333, 0.267     , 1.        ],
        [0.16666667, 0.3922    , 1.        ]]),
 array([[0.        , 0.4823    , 1.        ],
        [0.03333333, 0.1127    , 1.        ],
        [0.06666667, 0.4388    , 1.        ],
        [0.1       , 0.3631    , 1.        ],
        [0.13333333, 0.5605    , 1.        ]]),
 array([[0.        , 0.332

In [10]:
getMaxShape(details_preproc)

array([8, 6, 3])

In [11]:
details_ie.model.output_shape[-1]

8

In [12]:
b = details_ie.get_batch([0,1,2,8,9])

In [13]:
b[0][1]

array([[[0.        , 0.6415    , 1.        ],
        [0.03333333, 0.3793    , 1.        ],
        [0.06666667, 0.1965    , 1.        ],
        [0.1       , 0.2072    , 1.        ],
        [0.13333333, 0.6103    , 1.        ],
        [0.16666667, 0.2764    , 1.        ]],

       [[0.        , 0.1382    , 1.        ],
        [0.03333333, 0.4353    , 1.        ],
        [0.06666667, 0.4004    , 1.        ],
        [0.1       , 0.6575    , 1.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]],

       [[0.        , 0.6726    , 1.        ],
        [0.03333333, 0.6829    , 1.        ],
        [0.06666667, 0.121     , 1.        ],
        [0.1       , 0.4021    , 1.        ],
        [0.13333333, 0.267     , 1.        ],
        [0.16666667, 0.3922    , 1.        ]],

       [[0.        , 0.4823    , 1.        ],
        [0.03333333, 0.1127    , 1.        ],
        [0.06666667, 0.4388    , 1.        ],
        [0.1       , 0.3631 

In [14]:
transaction_data = [x['last_transactions'] for x in clientdata]
transaction_shape = (None, 3)

def transaction_preproc_function(x):
    #receives a dictionary with keys 'month' and 'value'
    #append a 1 to indicate that these values were not padded
    ans = []
    for item in x:
        ans.append([item['month']/12, item['value']/1000, 1])
    
    #handle case when list is empty
    if len(ans) == 0:
        ans.append([0,0,0])
    
    return np.array(ans)

inp = L.Input( (None,3+details_ie.model.output_shape[-1]) )

#inp = L.Input(transaction_shape)
x = L.LSTM(64)(inp)
x = L.Dense(16, activation='relu')(x)
x = L.Dense(8, activation='relu')(x)
transaction_model = Model(inputs=inp, outputs=x)
print('Original model')
transaction_model.summary()

transaction_ie = IntelligentElement(transaction_data, transaction_model, transaction_shape, children_ie=details_ie,
                                    preprocess_function=transaction_preproc_function, name='transaction_ie')

m, ii, oo = transaction_ie.retrieve_model_inputs_outputs()
print('\n\nRetrieved model')
m.summary()

Original model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None, 11)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                19456     
_________________________________________________________________
dense_3 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 136       
Total params: 20,632
Trainable params: 20,632
Non-trainable params: 0
_________________________________________________________________


Retrieved model
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inp_details_ie (InputLayer)     (None, 

In [15]:
transaction_ie.preprocess_function(transaction_ie.data[3])

array([[0.        , 0.002     , 1.        ],
       [0.08333333, 0.003     , 1.        ],
       [0.16666667, 0.004     , 1.        ],
       [0.25      , 0.003     , 1.        ],
       [0.33333333, 0.002     , 1.        ]])

In [16]:
bb = transaction_ie.get_batch([0,4,2,3,9])
print(len(bb)) #2 because there is input for the nested details element
bb[1][3]

2


array([[0.        , 0.002     , 1.        ],
       [0.08333333, 0.003     , 1.        ],
       [0.16666667, 0.004     , 1.        ],
       [0.25      , 0.003     , 1.        ],
       [0.33333333, 0.002     , 1.        ],
       [0.        , 0.        , 0.        ]])

In [17]:
#numerical for age and income, one-hot encoding for residence_region
client_data = [[x['age'], x['income'], x['residence_region']] for x in clientdata]
client_shape = (5,)
def client_preproc_function(x):
    #receives a list with age, income and residence_region
    idx = regions_id[x[2]]
    region_1hot = [0] * len(regions_id)
    region_1hot[idx] = 1

    return np.array([x[0]/100, np.log(1+x[1])]+region_1hot)
dim = get_children_sum_last_output_shapes([picture_ie,transaction_ie]) + client_shape[0]
print( dim )
inp=L.Input(  (dim,) )
            
x=inp
x=L.Dense(8, activation='relu')(x)
x=L.Dense(8, activation='relu')(x)
client_model = Model(inputs=inp, outputs=x)

print('Original model')
client_model.summary()

client_ie = IntelligentElement(client_data, client_model, client_shape, preprocess_function=client_preproc_function,
                               children_ie=[picture_ie, transaction_ie], name='client_ie')

m, ii, oo = client_ie.retrieve_model_inputs_outputs()
print('\n\nRetrieved model')
m.summary()

18
Original model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 18)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 152       
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 72        
Total params: 224
Trainable params: 224
Non-trainable params: 0
_________________________________________________________________


Retrieved model
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inp_details_ie (InputLayer)     (None, None, None, 3 0                                            
___________________________________________________________________________

In [18]:
client_ie.preprocess_function(client_ie.data[3])

array([0.32      , 3.04452244, 0.        , 1.        , 0.        ])

In [19]:
b = client_ie.get_batch([0,4,2,8,9])
print(b[0])
print(b[2][3])

[[[[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  ...

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]]


 [[[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  ...

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1